# Game Engine

In [ ]:
# | default_exp game.engine

In [ ]:
# | hide
%load_ext lab_black

In [ ]:
# | export
import importlib
import os

All games are located within the `games` subdirectory.

In [ ]:
# | export
TITLE_DIR = "engine/game/titles"
TITLE_MODULE = "rl18xx.game.engine.game.title"

In [ ]:
# | export


class Engine:
    def __init__(self):
        self.game_modules = self._import_game_modules()
        self.game_meta = self._collect_game_meta()
        self.games = self._load_games()

    def _import_game_modules(self):
        game_modules = {}
        for filename in os.listdir(TITLE_DIR):
            if filename.endswith(".ipynb"):
                module_name = filename[3:-6]
                module_path = f"{TITLE_MODULE}.{module_name}"
                game_modules[module_name] = importlib.import_module(module_path)
        return game_modules

    def _collect_game_meta(self):
        game_meta_by_title = {}
        for name, module in self.game_modules.items():
            if hasattr(module, "Meta"):
                meta = getattr(module, "Meta")
                game_meta_by_title[meta.title()] = meta
        return game_meta_by_title

    def _load_games(self):
        games = {}
        for title in self.game_meta:
            if hasattr(self.game_modules["g" + title], "Game"):
                games[title] = getattr(self.game_modules["g" + title], "Game")
        return games

    def meta_by_title(self, title):
        return self.game_meta.get(title)

    def game_by_title(self, title):
        return self.games.get(title)

In [ ]:
engine = Engine()

In [ ]:
game = engine.game_by_title("1830")

In [ ]:
g = game({"1": "hi", "2": "friend"})

In [ ]:
import inspect

skipped = []
methods = []
non_methods = []
for item_name in dir(g):
    if item_name.startswith("init") or item_name.startswith("_") or item_name.isupper():
        skipped.append(item_name)
        continue

    item = getattr(g, item_name)
    if callable(item):
        if inspect.signature(item).parameters:
            skipped.append(item)
        else:
            # print(item_name)
            methods.append(item_name)
            item()
    else:
        print(f"name: {item_name}, value: {item}")
        non_methods.append(item_name)

non_methods

name: actions, value: []
name: all_tiles, value: [<Tile: 1, hex: None>, <Tile: 2, hex: None>, <Tile: 3, hex: None>, <Tile: 3, hex: None>, <Tile: 4, hex: None>, <Tile: 4, hex: None>, <Tile: 7, hex: None>, <Tile: 7, hex: None>, <Tile: 7, hex: None>, <Tile: 7, hex: None>, <Tile: 8, hex: None>, <Tile: 8, hex: None>, <Tile: 8, hex: None>, <Tile: 8, hex: None>, <Tile: 8, hex: None>, <Tile: 8, hex: None>, <Tile: 8, hex: None>, <Tile: 8, hex: None>, <Tile: 9, hex: None>, <Tile: 9, hex: None>, <Tile: 9, hex: None>, <Tile: 9, hex: None>, <Tile: 9, hex: None>, <Tile: 9, hex: None>, <Tile: 9, hex: None>, <Tile: 14, hex: None>, <Tile: 14, hex: None>, <Tile: 14, hex: None>, <Tile: 15, hex: None>, <Tile: 15, hex: None>, <Tile: 16, hex: None>, <Tile: 18, hex: None>, <Tile: 19, hex: None>, <Tile: 20, hex: None>, <Tile: 23, hex: None>, <Tile: 23, hex: None>, <Tile: 23, hex: None>, <Tile: 24, hex: None>, <Tile: 24, hex: None>, <Tile: 24, hex: None>, <Tile: 25, hex: None>, <Tile: 26, hex: None>, <Tile: 27

['actions',
 'all_tiles',
 'bank',
 'cities',
 'closing_queue',
 'companies',
 'corporations',
 'corporations_are_closing',
 'current_entity',
 'depot',
 'exception',
 'filtered_actions',
 'final_turn',
 'finished',
 'game_companies',
 'game_corporations',
 'game_end_check_values',
 'game_market',
 'game_minors',
 'game_phases',
 'game_trains',
 'graph',
 'hexes',
 'id',
 'last_turn_start_action_id',
 'layout',
 'loading',
 'loans',
 'log',
 'minors',
 'names',
 'operating_rounds',
 'optional_rules',
 'phase',
 'players',
 'programmed_actions',
 'queued_log',
 'rand',
 'raw_actions',
 'raw_all_actions',
 'removals',
 'round',
 'round_counter',
 'round_history',
 'seed',
 'share_pool',
 'share_prices',
 'shares',
 'stock_market',
 'strict',
 'tile_groups',
 'tiles',
 'title',
 'total_loans',
 'trains',
 'turn',
 'undo_possible',
 'user']